In [11]:
import os

def check_missing_files(annotation_file, image_folder):
    missing_files = []
    
    with open(annotation_file, 'r') as file:
        for line in file:
            image_name = line.split()[0]
            image_path = os.path.join(image_folder, image_name)
            if not os.path.isfile(image_path):
                missing_files.append(image_name)
    
    if missing_files:
        print("Missing files:")
        for missing_file in missing_files:
            print(missing_file)
    else:
        print("All files are present.")

# Example usage
annotation_file = r'D:\Work\field-app\1main\Annote\yolo_annotate\training_data_ankit\training_data\Dryer_training\Prod Temp\annotations.txt'
image_folder = r'D:\Work\field-app\1main\Annote\yolo_annotate\training_data_ankit\training_data\Dryer_training\Prod Temp'
check_missing_files(annotation_file, image_folder)

All files are present.


In [12]:
def count_classes(annotation_file):
    class_counts = {}

    with open(annotation_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            for part in parts[1:]:
                class_label = part.split(',')[-1]
                if class_label in class_counts:
                    class_counts[class_label] += 1
                else:
                    class_counts[class_label] = 1

    print("Class counts:")
    for class_label, count in class_counts.items():
        print(f"Class {class_label}: {count}")

# Example usage
annotation_file = r'D:\Work\field-app\1main\Annote\yolo_annotate\training_data_ankit\training_data\Dryer_training\Prod Temp\annotations.txt'


count_classes(annotation_file)

Class counts:
Class 1: 42
Class 3: 65
Class 0: 22
Class 2: 86
Class 9: 84
Class 4: 23
Class 7: 9
Class 5: 31
Class 6: 15
Class 8: 4


In [14]:
import os

def delete_unlisted_files(annotation_file, image_folder):
    listed_files = set()
    
    # Read the annotation file and collect listed file names
    with open(annotation_file, 'r') as file:
        for line in file:
            image_name = line.split()[0]
            listed_files.add(image_name)
    
    # Collect .jpg files in the image folder that are not listed
    files_to_delete = []
    for file_name in os.listdir(image_folder):
        if file_name.endswith('.jpg') and file_name not in listed_files:
            files_to_delete.append(file_name)
    
    # Prompt for confirmation before deletion
    if files_to_delete:
        print(f"The following {len(files_to_delete)} .jpg files will be deleted:")
        for file_name in files_to_delete:
            print(file_name)
        
        confirm = input("Do you want to proceed with deletion? (yes/no): ")
        if confirm.lower() == 'yes':
            for file_name in files_to_delete:
                os.remove(os.path.join(image_folder, file_name))
            print("Files deleted.")
        else:
            print("Deletion aborted.")
    else:
        print("No unlisted .jpg files found.")

# Example usage
annotation_file = r'D:\Work\field-app\1main\Annote\yolo_annotate\training_data_ankit\training_data\Dryer_training\Prod Temp\annotations.txt'
image_folder = r'D:\Work\field-app\1main\Annote\yolo_annotate\training_data_ankit\training_data\Dryer_training\Prod Temp'
delete_unlisted_files(annotation_file, image_folder)

The following 15 .jpg files will be deleted:
20250215_005108.jpg
20250215_005130.jpg
20250215_005145.jpg
20250215_005201.jpg
20250215_005214.jpg
20250215_005229.jpg
20250215_005242.jpg
20250215_005256.jpg
20250215_005308.jpg
20250215_005322.jpg
20250215_005335.jpg
20250215_005405.jpg
20250215_005422.jpg
20250215_005538.jpg
20250215_005949.jpg
Files deleted.


In [18]:
import os
import cv2

# Paths for input and output
annotation_file = r'D:\Work\field-app\1main\Annote\yolo_annotate\training_data_ankit\training_data\Extruder_training\annotations.txt' # Replace with your path
images_output_dir = "dataset_15_feb_ank_3/train/images"
labels_output_dir = "dataset_15_feb_ank_3/train/labels"

os.makedirs(images_output_dir, exist_ok=True)
os.makedirs(labels_output_dir, exist_ok=True)

def process_annotations():
    with open(annotation_file, "r") as file:
        for line in file:
            # Split line and identify image path by .jpg
            parts = line.strip().split()
            img_path = None
            
            for i, part in enumerate(parts):
                if part.endswith(".jpg"):
                    img_path = " ".join(parts[:i + 1])
                    annotations = parts[i + 1:]
                    break
            
            if not img_path:
                print("No valid image path found in line.")
                continue

            frames_base_dir = r"D:\Work\field-app\1main\Annote\yolo_annotate\training_data_ankit\training_data\Extruder_training"
            img_path = os.path.join(frames_base_dir, img_path.strip())

            # Load image to get dimensions
            img = cv2.imread(img_path)
            if img is None:
                print(f"Image not found or unreadable: {img_path}")
                continue

            img_height, img_width = img.shape[:2]

            # Copy image to the target directory
            image_name = os.path.basename(img_path)
            target_image_path = os.path.join(images_output_dir, image_name)
            cv2.imwrite(target_image_path, img)

            # Prepare YOLO annotation file
            label_file_path = os.path.join(labels_output_dir, os.path.splitext(image_name)[0] + ".txt")
            with open(label_file_path, "w") as lbl_file:
                for annotation in annotations:
                    x1, y1, x2, y2, class_id = map(int, annotation.split(","))

                    # Normalize coordinates
                    x_center = ((x1 + x2) / 2) / img_width
                    y_center = ((y1 + y2) / 2) / img_height
                    width = (x2 - x1) / img_width
                    height = (y2 - y1) / img_height

                    # Write annotation in YOLO format
                    lbl_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

process_annotations()
print("Data successfully converted to YOLO format.")

Data successfully converted to YOLO format.
